<a href="https://colab.research.google.com/gist/MattPitlyk/45541145ad48b93da395f0a72ec2e7dc/fine-tuning-gpt-2-on-a-custom-dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning GPT-2 on a Custom Dataset

Reference: https://gist.github.com/MattPitlyk/45541145ad48b93da395f0a72ec2e7dc

In [ ]:
import pandas as pd


data = pd.read_excel('/content/full-foia-contacts.xls')
with open('/content/drive/MyDrive/my_custom_dataset.txt','a+', encoding='utf-8') as f:
    for line in data.values:
        f.write((str(line)+'\n'))

## Setup

In [ ]:
#read txt method two
f = open("/content/drive/MyDrive/my_custom_dataset.txt")
for line2 in f:
    print( line2)

流式输出内容被截断，只能显示最后 5000 行内容。
 'FOIA Public Liaison' nan nan nan nan nan '(202) 482-5992' nan nan nan

 nan nan]

['Office of the Secretary' 'Department of Commerce' 'Bobbie Parsons'

 'FOIA Officer, Immediate Office of the Secretary, Office of Privacy and Open Government'

 52010 '14th Street and Constitution Avenue, NW' 'Washington' 'DC' 20230

 '(202) 482-3257' '(202) 482-0827' 'mailto:Efoia@doc.gov'

 'http://www.osec.doc.gov/omo/FOIA/FOIAWebsite.htm' nan nan]

['Office of the Secretary' 'Department of Commerce' nan

 'FOIA Requester Service Center' nan nan nan nan nan '(202) 482-3257' nan

 nan nan nan nan]

['Office of the Secretary' 'Department of Commerce' 'Bobbie Parsons'

 'FOIA Public Liaison' nan nan nan nan nan '(202) 482-3257' nan nan nan

 nan nan]

["I don't know which office" 'Department of Commerce'

 'Michael Toland, Ph.D.'

 'Departmental FOIA Officer, Office of Privacy and Open Government' 52010

 '14th Street and Constitution Avenue, NW' 'Washington' 'DC' 20230

 '(2

Make sure you enable a GPU or TPU in the runtime  
Runtime -> Change Runtime -> Hardware Accelerator

Install helper library for fine-tuning.

In [ ]:
!pip install -q gpt-2-simple

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install tensorflow==1.15


ERROR: Could not find a version that satisfies the requirement tensorflow==1.15 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1)
ERROR: No matching distribution found for tensorflow==1.15


Imports

In [ ]:
# Colab pre-installs many common libraries including TensorFlow.
# Use Colab magic command to tell Colab to not use TensorFlow 2.0.
#%tensorflow_version 1.x
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

Download GPT-2 models.  
When fine-tuning on a single GPU, only the 124M and 335M size models can be used.

In [ ]:
gpt2.download_gpt2(model_name="124M")



Fetching checkpoint: 1.05Mit [00:00, 4.64Git/s]                                                     
Fetching encoder.json: 1.05Mit [00:00, 5.12Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 1.52Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:08, 61.0Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 2.10Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 6.44Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 7.07Mit/s]


Set variables

In [ ]:
file_name = "my_custom_dataset.txt"
run_name = 'fine_tuning_run_1'
model_size = '124M'

If your custom data is stored in your G-Drive, mount your drive and you can copy the data to Colab with the code below.  
Alternatively, you can upload your dataset directly to Colab using the Colab "Files" menu on the left (not the "File" menu above).  
Training examples in the dataset file should be separated with a blank line.

In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [ ]:
import pandas as pd


data = pd.read_excel('/content/full-foia-contacts.xls')
with open('/content/drive/MyDrive/my_custom_dataset.txt','a+', encoding='utf-8') as f:
    for line in data.values:
        f.write((str(line)+'\n'))

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Perform fine-tuning

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_size,
              steps=200,
              restore_from='fresh',
              run_name = run_name,
              print_every=10,
              sample_every=50,
              save_every=50,
	            only_train_transformer_layers = True,
	            accumulate_gradients = 1)
             # , learning_rate=.00003)

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.18it/s]


dataset has 111962 tokens
Training...
[10 | 9.75] loss=1.44 avg=1.44
[20 | 13.85] loss=1.90 avg=1.67
[30 | 18.06] loss=1.13 avg=1.49
[40 | 22.22] loss=1.20 avg=1.41
[50 | 26.38] loss=1.01 avg=1.33
Saving checkpoint/fine_tuning_run_1/model-50
======== SAMPLE 1 ========

Dennis' 'N. Dallin/E. F. Kennedy'
 'Suite 111-4-B' 'Suite 1007'
 'P.O. Box 089
 'Dallas' '' 'Dallas' '' 'F5J 4KD' '1001-A4'
 '914-344-0187'
 'http://www.dallaslive.com/FOIA.htm'
 'FOIA request submitted by N-B Railroad'
 'Dallas' 'FC'
 'Dallas' 'DFW' '7019F-5013'
 '915-344-0187'
 '714-344-0060'
 'http://www.dallaslive.com/FOIA.htm'
 'FOIA request provided by the DALLIN&FOIA Program'
 'Dallas' 'FC' 1119-4100
 'P.O. Box 089' 'Dallas' 'Dallas' 'F5J 4KD'
 '1001-A4' '914-344-0060'
 'http://www.dallaslive.com/FOIA.' 'FOIA request provided by the DALLIN&FOIA Program'
 'Dallas' 'FC'
 'Dallas' 'F5J 4KD' '1001-A4' '915-344-0060'
 'http://www.dallaslive.com/FOIA.' 'Signed-off-by' 'F-NAFAR' 'A9F 3B6'
 '505-719-5131' '(806) 639-5120'

Instructions for updating:
Use standard file APIs to delete files with this prefix.


======== SAMPLE 1 ========
'' 'mailto:crackett@msf.gov'
 'www.msf.gov' nan nan]
['Federal Emergency Management Agency' 'Department of Agriculture' nan
 'FOIA Requester Service Center' nan nan nan nan '(202) 734-8185' nan nan
 nan nan]
['Federal Emergency Management Agency' 'Department of Agriculture' nan
 'FOIA Public Liaison' nan nan nan nan nan '(202) 734-8145' nan
 'mailto:foiapost@bensa.bensa.gov'
 'http://www.bbc.bensa.gov/foia/policy/compliance/foia.shtml' nan
 nan]
['Federal Food, Nutrition, and Consumer Services Administration'
 'Department of Agriculture' nan
 'FOIA Requester Service Center' nan nan nan nan nan '(202) 734-0448' nan nan
 nan nan nan]
['Federal Food, Nutrition, and Consumer Services Administration'
 'Department of Agriculture' 'Steven Smith' 'FOIA Public Liaison' nan nan nan nan nan '(202) 734-2892' nan
 nan nan nan nan]
['Federal Bureau of Prisons' 'Department of Agriculture'
 'Amos Zalas' 'FOIA Contact' 'FOIA-APO'
 'BOP-SAC' '1345 S Fayetteville Pike' 'Atlanta

In [ ]:
# If G-Drive is mounted, save the fine-tuned model to the drive.
gpt2.copy_checkpoint_to_gdrive(run_name)

End of training

## Explore results

In [ ]:
print(run_name)

fine_tuning_run_1


In [ ]:
 # Copy the model from G-Drive if it wasn't trained in this Colab session.
gpt2.copy_checkpoint_from_gdrive(run_name=run_name)

In [ ]:
# Must reset the graph if training was done in this Colab session.
import tensorflow as tf
tf.compat.v1.reset_default_graph()

In [ ]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [ ]:
# Load the model.
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess,reuse=True, run_name=run_name)

Loading checkpoint checkpoint/fine_tuning_run_1/model-200


Unconditional generation

In [ ]:
#gpt2.generate(sess, run_name=run_name, temperature=.7, length=100, prefix=None, top_k=40, nsamples=10)

Conditional generation (give the model an input prompt)

In [ ]:
input_prompt = "Department of Veterans Affairs"

In [ ]:
gpt2.generate(sess, run_name=run_name, temperature=.7, length=100, prefix=input_prompt, top_k=40, nsamples=10)

Department of Veterans Affairs,' 'Department of Veterans Affairs'
 'Steven K. Covey' 'FOIA Public Liaison' nan nan nan nan nan nan '(202) 690-5453' nan
 nan nan nan nan nan]
['Office of Assistant Secretary for Administration and Management'
 'Department of Veterans Affairs' 'Susan H. Seegars' 'FOIA Contact'
 'Suite 6200' '300 E Street, SW' 'Washington' 'DC' 20210 '(202
Department of Veterans Affairs"
 "Department of Veterans Affairs' Legal Department" nan nan nan nan nan nan
 nan '(202) 616-3325' nan nan nan nan nan nan]
['Department of Veterans Affairs' 'Department of Veterans Affairs' 'Alesia Y. Williams'
 'FOIA Public Liaison' nan nan nan nan nan nan '(202) 616-3325' nan nan nan nan nan nan]
['Department of Veterans Affairs' 'Department of Veterans Affairs' '
Department of Veterans Affairs' 'Karen A. Jones'
 'FOIA Public Liaison' nan nan nan nan nan '(202) 622-4305' nan nan
 nan nan nan]
['Department of Veterans Affairs' 'Department of Veterans Affairs' 'Debbie Mills'
 'FOIA Request

## Copy model to Google Cloud Storage  
We have already copied the model to your G-Drive.  
This code will copy the model to your Google Cloud Platform Cloud Storage account (not free), where it can be used in data science deployment pipelines.


In [ ]:
project_id = 'your_gcp_project_id'
bucket_name = 'your_gcp_bucket_name'

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud config set project {project_id}

In [ ]:
# Need to zip just the inner folder.
# Currently, the name of the folder inside the zip needs to be `run1`.
!cd checkpoint; zip -r ../your_model_name.zip .

In [ ]:
!gsutil -m cp your_model_name.zip gs://{bucket_name}